In [0]:
"""
A simple notebook showing data analysis and data manipulation using pyspark.
Notebook imported from databricks cloud.
If needed, change file location to local directory under 'files' folder.
"""

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import col, lit, round, sum, avg, max, min, mean, count
spark = SparkSession.builder.appName("Office").getOrCreate()

In [0]:
file = "/FileStore/tables/OfficeDataProject-1.csv"
df = spark.read.options(InferSchema=True, header=True).csv(file)
df.show()
df.printSchema()

+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

In [0]:
# total number of employees

df.count()

Out[8]: 1000

In [0]:
# total number of departments

print(df.groupBy("department").count().count())
# or
df.select("department").dropDuplicates(["department"]).count()

6
Out[21]: 6

In [0]:
# number of employees by department

df.groupBy("department").count().show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|  169|
|        HR|  171|
|   Finance|  162|
|Purchasing|  166|
| Marketing|  170|
|  Accounts|  162|
+----------+-----+



In [0]:
# names of departments

df.select("department").dropDuplicates(["department"]).show()

+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



In [0]:
# number of employees in each state

df.groupBy("state").count().show()

+-----+-----+
|state|count|
+-----+-----+
|   LA|  205|
|   CA|  205|
|   WA|  208|
|   NY|  173|
|   AK|  209|
+-----+-----+



In [0]:
# number of employees in each state in each department

df.groupBy("department", "state").count().show()

+----------+-----+-----+
|department|state|count|
+----------+-----+-----+
|        HR|   WA|   47|
|        HR|   LA|   41|
| Marketing|   AK|   42|
| Marketing|   WA|   39|
|   Finance|   LA|   29|
|   Finance|   NY|   31|
|        HR|   NY|   30|
|  Accounts|   CA|   35|
| Marketing|   NY|   30|
|Purchasing|   CA|   32|
|     Sales|   CA|   42|
|     Sales|   LA|   35|
|        HR|   CA|   28|
| Marketing|   CA|   33|
|  Accounts|   AK|   37|
|Purchasing|   LA|   45|
|  Accounts|   NY|   34|
|     Sales|   WA|   27|
|        HR|   AK|   25|
|  Accounts|   WA|   27|
+----------+-----+-----+
only showing top 20 rows



In [0]:
# min and max salaries in each department, sorting ascending

df.groupBy("department").agg(min("salary").alias("min_salary"), max("salary").alias("max_salary")).orderBy(col("max_salary").asc()).show()

+----------+----------+----------+
|department|min_salary|max_salary|
+----------+----------+----------+
|  Accounts|      1007|      9890|
|   Finance|      1006|      9899|
| Marketing|      1031|      9974|
|        HR|      1013|      9982|
|     Sales|      1103|      9982|
|Purchasing|      1105|      9985|
+----------+----------+----------+



In [0]:
# printing the names of employees working in NY under Finance, whose bonuses are greater than avg bonuses of all NY employees

avg_bonus = df.filter(df.state == "NY").groupBy("state").agg(avg("bonus").alias("avg_bonus")).select("avg_bonus").collect()[0]["avg_bonus"]
df.filter((df.state == "NY") & (df.department == "Finance") & (df.bonus > avg_bonus)).show()

+-----------+--------------------+----------+-----+------+---+-----+
|employee_id|       employee_name|department|state|salary|age|bonus|
+-----------+--------------------+----------+-----+------+---+-----+
|       1035|       Vivan Sifford|   Finance|   NY|  1129| 35| 1261|
|       1073|      Herder Gallman|   Finance|   NY|  1988| 31| 1402|
|       1082|          Nena Rocha|   Finance|   NY|  3417| 25| 1647|
|       1087|       Leif Lemaster|   Finance|   NY|  8642| 45| 1782|
|       1100|Ellingsworth Meli...|   Finance|   NY|  7845| 32| 1358|
|       1127|        Escoto Gilma|   Finance|   NY|  3426| 41| 1285|
|       1161|     Georgeanna Laub|   Finance|   NY|  2469| 26| 1679|
|       1175|     Durio Tenenbaum|   Finance|   NY|  2253| 42| 1684|
|       1180|       Juliana Grigg|   Finance|   NY|  8178| 42| 1617|
|       1215|        Tiffani Benz|   Finance|   NY|  1665| 41| 1969|
|       1220|          Nitz Ilana|   Finance|   NY|  2443| 50| 1342|
|       1342|   Phylicia Antonina|

In [0]:
# raise salaries $500 of all employees whose age is greater than 45

def incr_salary(age, currentSalary):
    if age > 45:
        return currentSalary + 500
    return currentSalary

incrSalaryUDF = udf(lambda x,y: incr_salary(x,y), IntegerType())
df.withColumn("incr_salary", incrSalaryUDF(col("age"), col("salary"))).show()

+-----------+-------------------+----------+-----+------+---+-----+-----------+
|employee_id|      employee_name|department|state|salary|age|bonus|incr_salary|
+-----------+-------------------+----------+-----+------+---+-----+-----------+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|       6131|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|       4027|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|       3122|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|       6217|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|       5685|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|       2843|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|       7201|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|       3444|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|       4204|
|       1009|        Imai Hallie|  Accou